In [ ]:
import torch

print("PyTorch Version : {}".format(torch.__version__))

PyTorch Version : 1.12.1+cu113


In [ ]:
import torchvision

print("TorchVision Version : {}".format(torchvision.__version__))

TorchVision Version : 0.13.1+cu113


In [ ]:
import pycocotools
import numpy as np
from PIL import Image
from torchvision.transforms.functional import pil_to_tensor
from torchvision.utils import draw_bounding_boxes
from torchvision.transforms.functional import to_pil_image


In [ ]:
!wget http://images.cocodataset.org/annotations/annotations_trainval2017.zip
!unzip annotations_trainval2017.zip

--2022-11-16 02:55:13--  http://images.cocodataset.org/annotations/annotations_trainval2017.zip
Resolving images.cocodataset.org (images.cocodataset.org)... 54.231.162.233
Connecting to images.cocodataset.org (images.cocodataset.org)|54.231.162.233|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 252907541 (241M) [application/zip]
Saving to: ‘annotations_trainval2017.zip.1’

annotations_trainva 100%[===================>] 241.19M  82.0MB/s    in 2.9s    

2022-11-16 02:55:16 (82.0 MB/s) - ‘annotations_trainval2017.zip.1’ saved [252907541/252907541]

Archive:  annotations_trainval2017.zip
replace annotations/instances_train2017.json? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace annotations/instances_val2017.json? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace annotations/captions_train2017.json? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace annotations/captions_val2017.json? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace annotations/person_keypoints_train20

In [ ]:
from pycocotools.coco import COCO

annFile='annotations/instances_val2017.json'

coco=COCO(annFile)

# 1. Load Image


In [ ]:
def load_image(path):
  img = Image.open(path)
  width, height = img.size
  print(width, height)
  return img

def preproces(img):
  img_tensor_int = pil_to_tensor(img)
  img_tensor_int = img_tensor_int.unsqueeze(dim=0)
  img_tensor_float = img_tensor_int / 255.0
  return img_tensor_int, img_tensor_float


In [ ]:
# cm_far = load_image("/content/faraway_centralmarket.jpeg") #multiclass
# cm_far_tehuge_ppl = load_image("/content/huge_people.jpeg") #very high resolution
# huge_ppl_tensor_int, huge_ppl_tensor_float = preproces(huge_ppl)
# huge_pplnsor_int, cm_far_tensor_float = preproces(cm_far)
# cm_far

In [ ]:
# cm_close = load_image("/content/market-img2.jpeg") #multiclass
# cm_close_tensor_int, cm_close_tensor_float = preproces(cm_close)
# cm_close

In [ ]:
# huge_ppl = load_image("/content/huge_people.jpeg") #very high resolution
# huge_ppl_tensor_int, huge_ppl_tensor_float = preproces(huge_ppl)
# huge_ppl

In [1]:
# small_ppl = load_image("/content/small_people.jpeg") #very high resolution
# small_ppl_tensor_int, small_ppl_tensor_float = preproces(small_ppl)
# small_ppl

In [ ]:
# cm1 = load_image("/content/cm1.jpeg") #very high resolution
# cm1_tensor_int, cm1_tensor_float = preproces(cm1)
# cm1

In [5]:
# cm2 = load_image("/content/kina-QTYROea8lWs-unsplash.jpeg") #very high resolution
# cm2_tensor_int, cm2_tensor_float = preproces(cm2)
# cm2

In [ ]:
# cm3 = load_image("/content/kina-jchwccyZuhI-unsplash.jpeg") #very high resolution
# cm3_tensor_int, cm3_tensor_float = preproces(cm3)
# cm3

In [ ]:
# cm4 = load_image("/content/kina-jchwccyZuhI-unsplash.jpeg") #very high resolution
# cm4_tensor_int, cm4_tensor_float = preproces(cm4)
# cm4

In [6]:
# cm5 = load_image("/content/45864770764_807afc5ef2_k.jpeg") #very high resolution
# cm5_tensor_int, cm5_tensor_float = preproces(cm5)
# cm5

# 2. Load Pre-Trained Model

In [ ]:
from torchvision.models.detection import fasterrcnn_resnet50_fpn,ssd300_vgg16

frnn_model = fasterrcnn_resnet50_fpn(pretrained=True, progress=False)
ssd_model = ssd300_vgg16(pretrained=True, progress=False)

/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:209: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  f"The parameter '{pretrained_param}' is deprecated since 0.13 and will be removed in 0.15, "
/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=FasterRCNN_ResNet50_FPN_Weights.COCO_V1`. You can also use `weights=FasterRCNN_ResNet50_FPN_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=SSD300_VGG16_Weights.COCO_V1`. Y

In [ ]:
def pred(img_tensor_float, th, algo):
  if algo == "FRNN":
    frnn_model.eval(); ## Setting Model for Evaluation/Prediction

    img_preds = frnn_model(img_tensor_float)
    img_preds[0]["boxes"] = img_preds[0]["boxes"][img_preds[0]["scores"] > th]
    img_preds[0]["labels"] = img_preds[0]["labels"][img_preds[0]["scores"] > th]
    img_preds[0]["scores"] = img_preds[0]["scores"][img_preds[0]["scores"] > th]
    # Map Target Category Ids to Labels
    img_labels = coco.loadCats(img_preds[0]["labels"].numpy())

  elif algo == "SSD": 
    ssd_model.eval(); ## Setting Model for Evaluation/Prediction

    img_preds = ssd_model(img_tensor_float)
    img_preds[0]["boxes"] = img_preds[0]["boxes"][img_preds[0]["scores"] > th]
    img_preds[0]["labels"] = img_preds[0]["labels"][img_preds[0]["scores"] > th]
    img_preds[0]["scores"] = img_preds[0]["scores"][img_preds[0]["scores"] > th]
    # Map Target Category Ids to Labels
    img_labels = coco.loadCats(img_preds[0]["labels"].numpy())
  return img_labels,img_preds

In [ ]:
def bounding_box(img,th=0.8,algo="FRNN"): #"FRNN | SSD"
  img_tensor_int, img_tensor_float = preproces(img)
  img_labels, img_preds = pred(img_tensor_float,th,algo)
  img_annot_labels = ["{}-{:.2f}".format(label["name"], prob) for label, prob in zip(img_labels, img_preds[0]["scores"].detach().numpy())]

  img_output = draw_bounding_boxes(image=img_tensor_int[0],
                             boxes=img_preds[0]["boxes"],
                             labels=img_annot_labels,
                             colors=["red" if label["name"]=="person" else "green" for label in img_labels],
                             width=2
                            )
  print(img_output.shape)
  return to_pil_image(img_output)


In [ ]:

import matplotlib.pyplot as plt

def show(img, figsize):
    npimg = img.numpy()
    plt.figure(figsize=figsize, dpi=80)
    plt.imshow(np.transpose(npimg, (1, 2, 0)), interpolation='nearest')
    plt.show()

def bounding_box_plt(img,th=0.8,algo="FRNN", size=(10,10)): #"FRNN | SSD"
  img_tensor_int, img_tensor_float = preproces(img)
  img_labels, img_preds = pred(img_tensor_float,th,algo)
  img_annot_labels = ["{}-{:.2f}".format(label["name"], prob) for label, prob in zip(img_labels, img_preds[0]["scores"].detach().numpy())]

  img_output = draw_bounding_boxes(image=img_tensor_int[0],
                             boxes=img_preds[0]["boxes"],
                             labels=img_annot_labels,
                             colors=["red" if label["name"]=="person" else "green" for label in img_labels],
                             width=2
                            )
  print(img_output.shape)
  return show(img_output, figsize=size)

# 3. Visualize

In [ ]:
# cm1_output_ssd = bounding_box(cm1,th=0.7,algo="SSD")
# cm1_output_ssd

In [ ]:
# cm1_output_frnn = bounding_box(cm1,th=0.7,algo="FRNN")
# cm1_output_frnn

In [7]:
# cm2_output_frnn = bounding_box(cm2,th=0.7,algo="FRNN")
# cm2_output_frnn

In [8]:
# cm2_output_ssd = bounding_box_plot(cm2,th=0.7,algo="SSD")
# cm2_output_ssd

In [9]:
# cm5_output_frnn = bounding_box(cm5,th=0.7,algo="FRNN")
# cm5_output_frnn

In [10]:
# cm5_output_ssd = bounding_box(cm5,th=0.7,algo="SSD")
# cm5_output_ssd

In [ ]:
# cm3_output_frnn = bounding_box(cm3,th=0.7,algo="FRNN")
# cm3_output_frnn

In [ ]:
# cm3_output_ssd = bounding_box(cm3,th=0.7,algo="SSD")
# cm3_output_ssd

In [ ]:
# cm_close_output_ssd = bounding_box_plt(cm_close,th=0.7,algo="SSD", size=(10,10))

In [ ]:
# cm_close_output_frnn = bounding_box_plt(cm_close,th=0.7,algo="FRNN", size=(10,10))

In [ ]:
# cm_far_output_ssd = bounding_box_plt(cm_far,th=0.7,algo="SSD", size=(10,10))

In [ ]:
# cm_far_output_frnn = bounding_box_plt(cm_far,th=0.7,algo="FRNN", size=(10,10))

In [ ]:
# huge_ppl_output_frnn = bounding_box_plt(huge_ppl,th=0.7,algo="FRNN", size=(10,10))

In [ ]:
# huge_ppl_output_ssd = bounding_box_plt(huge_ppl,th=0.7,algo="SSD", size=(10,10))

In [3]:
# small_ppl_output_frnn = bounding_box(small_ppl,th=0.7,algo="FRNN")
# small_ppl_output_frnn

In [4]:
# small_ppl_output_ssd = bounding_box(small_ppl,th=0.7,algo="SSD")
# small_ppl_output_ssd

In [ ]:
# import torch

# # Model
# model = torch.hub.load('ultralytics/yolov5', 'yolov5s')
# model.conf=0.6
# # # Image
# # img = cv2.imread("/content/centra.webp") # Read image with cv2
# # img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB) 
# # # im = 'https://ultralytics.com/images/zidane.jpg'

# # Inference
# results = model(small_ppl)

# results.show()

In [ ]:
# results = model(huge_ppl)

# results.show()